In [ ]:
!pip install pandas
!pip install pyspark
!pip install pyarrow

In [ ]:
from pyspark.sql import SparkSession
 
# Building the SparkSession and name 
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [3]:
import pandas as pd

data = [[6, 'Alice'], [2, 'Bob'], [7, 'Alex']]
users = pd.DataFrame(data, columns=['user_id', 'user_name']).astype({'user_id':'Int64', 'user_name':'object'})
data = [[215, 6], [209, 2], [208, 2], [210, 6], [208, 6], [209, 7], [209, 6], [215, 7], [208, 7], [210, 2], [207, 2], [210, 7]]
register = pd.DataFrame(data, columns=['contest_id', 'user_id']).astype({'contest_id':'Int64', 'user_id':'Int64'})

In [4]:
users = spark.createDataFrame(users)
users.show()

register = spark.createDataFrame(register)
register.show()

+-------+---------+
|user_id|user_name|
+-------+---------+
|      6|    Alice|
|      2|      Bob|
|      7|     Alex|
+-------+---------+

+----------+-------+
|contest_id|user_id|
+----------+-------+
|       215|      6|
|       209|      2|
|       208|      2|
|       210|      6|
|       208|      6|
|       209|      7|
|       209|      6|
|       215|      7|
|       208|      7|
|       210|      2|
|       207|      2|
|       210|      7|
+----------+-------+



In [8]:
from pyspark.sql.functions import count, col, round

register \
.groupby('contest_id') \
.agg(count('user_id').alias('contestants')) \
.withColumn('percentage', round(col('contestants') / users.count() * 100, 2)) \
.select(['contest_id', 'percentage']) \
.show()

+----------+----------+
|contest_id|percentage|
+----------+----------+
|       215|     66.67|
|       209|     100.0|
|       208|     100.0|
|       210|     100.0|
|       207|     33.33|
+----------+----------+

